# Neural Networks Final Project
### Reimplementation of the study: <br> ***"DE-FAKE: Detection and Attribution of Fake Images Generated by Text-to-Image GenerationModels"* <br> from Zeyang Sha, Zheng Li, Ning Yu, Yang Zhang**

**Name**: *Laura Papi*

**Matricola**: *1760732*

# Project Description

The above cited study focuses on the growing concerns about the possible misuse of AI generated images, and assesses the necessity for a tool to detect, and attribute, these fake images.<br>
In particular, it points out the lack of research on the particular case of images generated by a text prompt.
<br>

<br>
Therefore, this research proposes methods to answer the following 3 research questions [RQ]:

- **RQ1**. Detection of images generated by text-to-image generation models

- **RQ2**. Attribution of the fake images to their source model

- **RQ3**. Analysis of the likelihood that different text prompts have to generate authentic images

<br><br>
The following sections contain examples for my implementation of the described methods.<br><br>
The complete implementation of the models can be found in the source directory of the GitHub repository __[Source Code](http://url)__


In [ ]:
# -- Declare the variables to be used globally in this notebook

path_to_ld = "../latent-diffusion/" # set here the path to the LD directory cloned from GitHub
proj_dir = "../De-Fake_nn_final_project" # set here the path to the root of the current project (De-Fake)

SD_generated_temp_dir = "data/generated/SD+MSCOCO/"
GLIDE_generated_temp_dir = "data/generated/GLIDE+MSCOCO/"
LD_generated_temp_dir = path_to_ld + "outputs/txt2img-samples/"

In [ ]:
# -- Declare all the imports needed in this notebook

# External libraries imports
import sys
import os
import torch

# References to other files of this project
from src.data_collector import fetchImagesFromMSCOCO
from src.dataset_generator import SD_generation, LD_generation, GLIDE_generation
from src.format_dataset import format_dataset_binaryclass, formatIntoTrainTest, format_dataset_multiclass
from src.encoder import get_multiclass_dataset_loader

from src.hybrid_attributor.model import MultiClassTwoLayerPerceptron, train_hybrid_attributor


## RQ1. Detection of images generated by text-to-image generation models

The study proposes two detector models:

1. **Image-only detector**<br>binary classifier that decides whether an input image is fake or real.

2. **Hybrid detector**<br>binary classifier that is able to tell if an image is fake or real, based on the input image and its corresponding text prompt.


### 1. Image-only detector

#### 1.1 Dataset
All the datasets are constitueted by a set of N real images (labeled 1), and a set of N corresponding fake generated images (labeled 0).

Training (on a single dataset):
- real images fetched from MSCOCO
- fake images generated by Stable Diffusion (SD)

Evaluation (on three different datasets):
- real images always fetched from MSCOCO
- fake images generated respectively by Stable Diffusion (SD), Latent Diffusion (LD) and GLIDE


##### 1.1.1 Data Collection

- **(i)** Real images are fetched from the MSCOCO dataset, together with their captions.

In [ ]:
#import the path to the scripts needed for this section
sys.path.insert(10, '/home/parwal/Documents/GitHub/De-Fake_nn_final_project/src/imageonly_detector')
#TODO capire a chi serve questo import e metterlo nel posto giusto

# TODO TEST

#SD+MSCOCO
# The dataset generated using SD will be divided into train and test later on
fetchImagesFromMSCOCO("data/MSCOCO_for_SD/images", "data/MSCOCO_for_SD", 100)

#LD+MSCOCO --------------------------------------------------------------------------
# real images and captions needed as input for the LD model
fetchImagesFromMSCOCO("data/imageonly_detector_data/val_LD/class_1", "data/imageonly_detector_data/val_LD", 50)

#GLIDE+MSCOCO -----------------------------------------------------------------------
# real images and captions needed as input for the GLIDE model
fetchImagesFromMSCOCO("data/imageonly_detector_data/val_GLIDE/class_1", "data/imageonly_detector_data/val_GLIDE", 50)

- **(ii)** The captions from the MSCOCO images are used as input to the text-to-image generator models (SD, LD, GLIDE).

In [ ]:
# TODO TEST

#SD+MSCOCO --------------------------------------------------------------------------
#use stable-diffusion API to generate 100 fake images from the 100 captions collected before
#prima di eseguire il file ho cambiato le directory
#%run src/imageonly_detector/SD_MSCOCO_data_generation.py
SD_generation("data/MSCOCO_for_SD/mscoco_captions.csv", SD_generated_temp_dir)

#LD+MSCOCO --------------------------------------------------------------------------
#resetto la directory corrente a quella del progetto de-fake, altrimenti il file da eseguire non viene trovato
#questo è necessario perché LD_MSCOCO_data_generation.py cambia la directory a quella di latent-diffusion
#os.chdir("/home/parwal/Documents/GitHub/De-Fake_nn_final_project")
#%run src/imageonly_detector/LD_MSCOCO_data_generation.py
LD_generation("data/imageonly_detector_data/val_LD/mscoco_captions.csv")

#GLIDE+MSCOCO -----------------------------------------------------------------------
#%run src/imageonly_detector/GLIDE_MSCOCO_data_generation.ipynb
GLIDE_generation("data/imageonly_detector_data/val_GLIDE/mscoco_captions.csv", GLIDE_generated_temp_dir)

##### 1.1.2 Dataset Construction
The collected and generated data is then shaped in the following structure, in order to be used for the training and evaluation step:<br><br>
train/<br>
    ├── class_0/<br>
    │   ├── ...<br>
    │   └── all the fake images<br>
    ├── class_1/<br>
    │   ├── ...<br>
    │   └── all the real images<br>
val/<br>
    ├── class_0/<br>
    │   ├── ...<br>
    │   └── all the fake images<br>
    ├── class_1/<br>
    │   ├── ...<br>
    │   └── all the real images<br>

In [ ]:
#transform the collected data in the previously described structure

# TODO TEST

#SD+MSCOCO
#this function generates a pair of datasets (train and val), starting from data from the Stable Diffusion generation
#the data generated from SD contains 100 images, this original dataset is split in half (50 for train, 50 for test)
formatIntoTrainTest("data/MSCOCO_for_SD/images", SD_generated_temp_dir, "data/imageonly_detector_data")
print("ok SD")

#LD+MSCOCO --------------------------------------------------------------------------
format_dataset_binaryclass(LD_generated_temp_dir, "data/imageonly_detector_data/val_LD")
print("ok LD")

#GLIDE+MSCOCO -----------------------------------------------------------------------
format_dataset_binaryclass(GLIDE_generated_temp_dir, "data/imageonly_detector_data/val_GLIDE")
print("ok GLIDE")

#### 1.2 Model

The model is defined and trained in the file executed in the followind code block.

In [ ]:
#this function trains the model and tests it at every epoch
#both the test and train datasets are generated using SD
%run src/imageonly_detector/train.py

### 2. Hybrid detector

#### 2.1 Dataset

The dataset is built in the exact same way as the dataset for the image-only detector.<br>
Run the following blocks to build the dataset for the hybrid detector.

In [ ]:
# ------------------- COLLECT REAL IMAGES FROM MSCOCO -------------------- #

# TODO TEST

#SD+MSCOCO
fetchImagesFromMSCOCO("data/MSCOCO_for_SD_hybrid/images", "data/MSCOCO_for_SD_hybrid", 100)

#LD+MSCOCO --------------------------------------------------------------------------
fetchImagesFromMSCOCO("data/hybrid_detector_data/val_LD/class_1", "data/hybrid_detector_data/val_LD", 50)

#GLIDE+MSCOCO -----------------------------------------------------------------------
fetchImagesFromMSCOCO("data/hybrid_detector_data/val_GLIDE/class_1", "data/hybrid_detector_data/val_GLIDE", 50)

In [ ]:
# ------------------- GENERATE FAKE IMAGES USING SD, LD, GLIDE -------------------- #
#SD+MSCOCO --------------------------------------------------------------------------
#use stable-diffusion API to generate 100 fake images from the 100 captions collected before
#%run src/imageonly_detector/SD_MSCOCO_data_generation.py
SD_generation("data/MSCOCO_for_SD_hybrid/mscoco_captions.csv", SD_generated_temp_dir)

#LD+MSCOCO --------------------------------------------------------------------------
# N.B.
# prima di lanciare questo comando, aggiungere il file src/imageonly_detector/txt2img_batch.py alla directory latent-diffusion/scripts/
#resetto la directory corrente a quella del progetto de-fake, altrimenti il file da eseguire non viene trovato
#questo è necessario perché LD_MSCOCO_data_generation.py cambia la directory a quella di latent-diffusion
#os.chdir("/home/parwal/Documents/GitHub/De-Fake_nn_final_project")
#%run src/imageonly_detector/LD_MSCOCO_data_generation_batch.py
LD_generation("data/hybrid_detector_data/val_LD/mscoco_captions.csv")

#GLIDE+MSCOCO -----------------------------------------------------------------------
#NON HO MAI PROVATO A RUNNARLO, altrimenti rigenera il modello (3gb)
#provare a runnarlo proprio alla fine di tutto per sicurezza
#%run src/imageonly_detector/GLIDE_MSCOCO_data_generation.ipynb #TODO
GLIDE_generation("data/hybrid_detector_data/val_GLIDE/mscoco_captions.csv", GLIDE_generated_temp_dir)

In [ ]:
# ------------------- FORMAT THE DATA INTO THE STRUCTURE NEEDED FOR TRAINING/TESTING -------------------- #
os.chdir("/home/parwal/Documents/GitHub/De-Fake_nn_final_project")

#transform the collected data in the previously described structure

#SD+MSCOCO --------------------------------------------------------------------------
#this function generates a pair of datasets (train and val), starting from data from the Stable Diffusion generation
#the data generated from SD contains 100 images, this original dataset is split in half (50 for train, 50 for test)
formatIntoTrainTest("data/MSCOCO_for_SD_hybrid/images", SD_generated_temp_dir, "data/hybrid_detector_data")
print("ok SD")

#LD+MSCOCO --------------------------------------------------------------------------
format_dataset_binaryclass(LD_generated_temp_dir, "data/hybrid_detector_data/val_LD")
print("ok LD")

#GLIDE+MSCOCO -----------------------------------------------------------------------
format_dataset_binaryclass(GLIDE_generated_temp_dir, "data/hybrid_detector_data/val_GLIDE")
print("ok GLIDE")

#### 2.2 Model

The model structure is defined in src/hybrid_detector/hybrid_detector.py<br><br>

The training dataset is generated using Stable Diffusion (SD), same as for the image-only detector.<br>
It can be trained running the following command:

In [ ]:
# this function first builds the dataloader for the training datasert
# then it trains the model and saves the trained weights in the directory trained_models/
%run src/hybrid_detector/train.py

Now that the model is trained, we can evaluate it on some test datasets.<br>
In particular we will evaluate it on:<br>
- Stable Diffusion (SD), dataset generated from the same image-to-text generator used for the train dataset.
- GLIDE
- Latent Diffusion

In [ ]:
# encode the data into dataloaders suitable for the model, and run the evaluation algorithm.
%run src/hybrid_detector/eval.py

## RQ2. Attribution of the fake images to their source model

The study proposes two attributor models:

1. **Image-only attributor**<br>multi-class classifier that assigns each input image to its source generation model, given the image only.

2. **Hybrid attributor**<br>multi-class classifier that assigns each input image to its source generation model, based on the input image and its corresponding text prompt.


### 1. Image-only attributor

In this section we will build and train a model that is able to assign an image to the model that generated it, given only that image.<br><br>
The classes that this model will be able to address are the following:
- real image -> class 0
- fake image generated by SD -> class 1
- fake image generated by LD -> class 2
- fake image generated by GLIDE -> class 3

#### 1.1 Dataset

We generate two datasets, one for training and one for evaluating the model.<br>
The steps needed to generate the two datasets are the same:
- fetch real images and their captions from MSCOCO (class 0)
- generate fake images with SD using the captions of the real images (class 1)
- generate fake images with LD using the captions of the real images (class 2)
- generate fake images with GLIDE using the captions of the real images (class 3)
- move the real and generated images into a dataset directory, with the following structure:

imageonly_attributor_data/<br>
&emsp;&emsp;├── train/<br>
&emsp;&emsp;&emsp;&emsp;├── class_real/<br>
&emsp;&emsp;&emsp;&emsp;│   ├── ...<br>
&emsp;&emsp;&emsp;&emsp;│   └── *all the images fetched by MSCOCO*<br>
&emsp;&emsp;&emsp;&emsp;├── class_SD/<br>
&emsp;&emsp;&emsp;&emsp;│   ├── ...<br>
&emsp;&emsp;&emsp;&emsp;│   └── *all the images generated by SD*<br>
&emsp;&emsp;&emsp;&emsp;├── class_GLIDE/<br>
&emsp;&emsp;&emsp;&emsp;│   ├── ...<br>
&emsp;&emsp;&emsp;&emsp;│   └── *all the images generated by GLIDE*<br>
&emsp;&emsp;&emsp;&emsp;├── class_LD/<br>
&emsp;&emsp;&emsp;&emsp;│   ├── ...<br>
&emsp;&emsp;&emsp;&emsp;│   └── *all the images generated by LD*<br>
&emsp;&emsp;├── test/<br>
&emsp;&emsp;&emsp;&emsp;├── ...<br>

Generate the TRAIN dataset first:

In [ ]:
os.chdir(proj_dir)

# fetch the images with their captions from MSCOCO (N=50)
fetchImagesFromMSCOCO("data/imageonly_attributor_data/train/class_real", "data/imageonly_attributor_data/train", 50)

# use the same 50 captions to generate images with SD
SD_generation("data/imageonly_attributor_data/train/mscoco_captions.csv", SD_generated_temp_dir)

# use the same 50 captions to generate images with GLIDE
GLIDE_generation("data/imageonly_attributor_data/train/mscoco_captions.csv", GLIDE_generated_temp_dir)

# use the same 50 captions to generate images with LD
LD_generation("data/imageonly_attributor_data/train/mscoco_captions.csv")

# move the generated images to the dataset dir
format_dataset_multiclass(SD_generated_temp_dir, LD_generated_temp_dir, GLIDE_generated_temp_dir, "data/imageonly_attributor_data/train")

Then generate the TEST dataset:

In [ ]:
# Repeat the same procedure for the test dataset

# fetch the images with their captions from MSCOCO (N=50)
fetchImagesFromMSCOCO("data/imageonly_attributor_data/test/class_real", "data/imageonly_attributor_data/test", 50)

# use the same 50 captions to generate images with SD
SD_generation("data/imageonly_attributor_data/test/mscoco_captions.csv", SD_generated_temp_dir)

# use the same 50 captions to generate images with GLIDE
GLIDE_generation("data/imageonly_attributor_data/test/mscoco_captions.csv", GLIDE_generated_temp_dir)

# use the same 50 captions to generate images with LD
LD_generation("data/imageonly_attributor_data/test/mscoco_captions.csv")

# move the generated images to the dataset dir
format_dataset_multiclass(SD_generated_temp_dir, LD_generated_temp_dir, GLIDE_generated_temp_dir, "data/imageonly_attributor_data/test")

#### 1.2 Model

In [ ]:
#this function trains the model and tests it at every epoch
#both the test and train datasets are generated using MSCOCO, SD, LD, GLIDE
%run src/imageonly_attributor/attributor.py

### 2. Hybrid attributor
In this section we will build and train a model similar to the model built in section 1.<br>
The difference is that instead of taking as input only the image, this model also considers its textual caption.

#### 2.1 Dataset

Train and test datasets are generated in the same way as in the image-only attributor case.<br>
For the dataset directory structure also refer to the previous section.

Generate the TRAIN dataset first:

In [ ]:
# fetch the images with their captions from MSCOCO (N=50)
fetchImagesFromMSCOCO("data/hybrid_attributor_data/train/class_real", "data/hybrid_attributor_data/train", 50)

# use the same 50 captions to generate images with SD
SD_generation("data/hybrid_attributor_data/train/mscoco_captions.csv", SD_generated_temp_dir)

# use the same 50 captions to generate images with GLIDE
GLIDE_generation("data/hybrid_attributor_data/train/mscoco_captions.csv", GLIDE_generated_temp_dir)

# use the same 50 captions to generate images with LD OK
LD_generation("data/hybrid_attributor_data/train/mscoco_captions.csv")

# move the generated images to the dataset dir
format_dataset_multiclass(SD_generated_temp_dir, LD_generated_temp_dir, GLIDE_generated_temp_dir, "data/hybrid_attributor_data/train")

Then denerate the TEST dataset:

In [ ]:
# fetch the images with their captions from MSCOCO (N=50)
fetchImagesFromMSCOCO("data/hybrid_attributor_data/test/class_real", "data/hybrid_attributor_data/test", 50)

# use the same 50 captions to generate images with SD
SD_generation("data/hybrid_attributor_data/test/mscoco_captions.csv", SD_generated_temp_dir)

# use the same 50 captions to generate images with GLIDE
GLIDE_generation("data/hybrid_attributor_data/test/mscoco_captions.csv", GLIDE_generated_temp_dir)

# use the same 50 captions to generate images with LD OK
LD_generation("data/hybrid_attributor_data/test/mscoco_captions.csv")

# move the generated images to the dataset dir
format_dataset_multiclass(SD_generated_temp_dir, LD_generated_temp_dir, GLIDE_generated_temp_dir, "data/hybrid_attributor_data/test")

#### 2.2 Model

In the next block we build and train the actual model, a two-layer perceptron for multiclass classification, with the following steps:
- **Build the model** using a custom implemented module<br>A two-layer perceptron that outputs the class predicted for each sample<br><br>
- **Create Dataset and DataLoader** objects starting from the row data fetched at 2.1 (jpg images and string captions)<br>Each item of this dataset is composed by the encoding of an image concatenated with the encoding of its caption,<br>the encodings are generated using the CLIP model<br><br>
- **Train the model** using a custom train function and the DataLoader from the previous step

In [1]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Build the model
print('Building the model...')
hybrid_attributor = MultiClassTwoLayerPerceptron(1024, 100, 4).to(device)

# Build the dataset (each sample in the dataset is the encoding of an image concatenated to the encoding of its caption - encodings generated using the CLIP model)
print('Building the dataset...')
captions_file = "data/hybrid_attributor_data/train/mscoco_captions.csv"
dataset_dir = "data/hybrid_attributor_data/train"
classes = {"class_real", "class_SD", "class_LD", "class_GLIDE"}

train_data_loader = get_multiclass_dataset_loader(captions_file, dataset_dir, classes)

# Train the model on the dataset just generated
print('Training starts:')
train_hybrid_attributor(hybrid_attributor, train_data_loader, 30, 0.005)

Building the model...
Building the dataset...
Training starts:
epoch: 1/30
EPOCH:  1  - MEAN ACCURACY:  tensor(0.2200)  - MEAN LOSS:  tensor(1.3872)
epoch: 2/30
EPOCH:  2  - MEAN ACCURACY:  tensor(0.3183)  - MEAN LOSS:  tensor(1.3617)
epoch: 3/30
EPOCH:  3  - MEAN ACCURACY:  tensor(0.4183)  - MEAN LOSS:  tensor(1.3365)
epoch: 4/30
EPOCH:  4  - MEAN ACCURACY:  tensor(0.5933)  - MEAN LOSS:  tensor(1.3108)
epoch: 5/30
EPOCH:  5  - MEAN ACCURACY:  tensor(0.6767)  - MEAN LOSS:  tensor(1.2796)
epoch: 6/30
EPOCH:  6  - MEAN ACCURACY:  tensor(0.7567)  - MEAN LOSS:  tensor(1.2472)
epoch: 7/30
EPOCH:  7  - MEAN ACCURACY:  tensor(0.7767)  - MEAN LOSS:  tensor(1.2103)
epoch: 8/30
EPOCH:  8  - MEAN ACCURACY:  tensor(0.7817)  - MEAN LOSS:  tensor(1.1686)
epoch: 9/30
EPOCH:  9  - MEAN ACCURACY:  tensor(0.8517)  - MEAN LOSS:  tensor(1.1257)
epoch: 10/30
EPOCH:  10  - MEAN ACCURACY:  tensor(0.7917)  - MEAN LOSS:  tensor(1.0752)
epoch: 11/30
EPOCH:  11  - MEAN ACCURACY:  tensor(0.8250)  - MEAN LOSS:  te

## RQ3. Analysis of the likelihood that different text prompts have to generate authentic images

### 1. Semantic Analysis

### 2. Structure Analysis

## Conclusions